In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/bureau-and-balance/bureau (1).csv


In [2]:
df_bureau = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau.csv")
df_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau_balance.csv")
print(df_bureau.shape)
print(df_balance.shape)


(1716428, 17)
(27299925, 3)


In [3]:
df_bureau

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [4]:
categorical_columns = df_bureau.select_dtypes(exclude = 'number').columns
categorical_bureau = df_bureau[categorical_columns].copy()
categorical_values = categorical_bureau.nunique()

In [5]:
label = []
hot = []

for i in range(0,len(categorical_values)):
    if categorical_values.iloc[i]>2:
        hot.append(categorical_values.index[i])
    else:
        label.append(categorical_values.index[i])

In [6]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

hot_encoder = OneHotEncoder(handle_unknown = 'ignore',sparse_output=False)
encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value',unknown_value = -1)

hot_encoded_features = pd.DataFrame(hot_encoder.fit_transform(categorical_bureau[hot]),columns=hot_encoder.get_feature_names_out(hot))
ordinal_encoded_features = pd.DataFrame(encoder.fit_transform(categorical_bureau[label]),columns=label)

df_categorical = pd.concat([hot_encoded_features,ordinal_encoded_features],axis=1)
df_categorical

,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,...,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1716424,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1716425,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1716426,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
numerical_columns = df_bureau.select_dtypes(include='number').columns
df_numerical = df_bureau[numerical_columns].copy()

df = pd.concat([df_numerical,df_categorical],axis=1)
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.50,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1716424,100044,5057754,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1716425,100044,5057762,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1716426,246829,5057770,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
empty_columns=[]

for column in df.columns:
    if df[column].isnull().sum()/305811>0.9:
        empty_columns.append(column)

df= df.drop(columns=empty_columns,axis=1)

print(empty_columns)
print(len(empty_columns))


['DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_LIMIT', 'AMT_ANNUITY']
4


In [24]:
# Manual feature engineering

df['PERCENTAGE_REIMBURSED'] = df['AMT_CREDIT_SUM_DEBT']/df['AMT_CREDIT_SUM']
df['NUMBER_LOANS'] = df.groupby('SK_ID_CURR')['SK_ID_CURR'].transform('count')

In [69]:
df_aggregated = df.copy()
df_id = df_aggregated.groupby(['SK_ID_CURR'],as_index=False)['SK_ID_BUREAU'].first()
df_aggregated = df_aggregated.groupby(['SK_ID_CURR'],as_index=False).agg(['mean','count','sum','max','min'])
df_aggregated = df_aggregated.drop('SK_ID_BUREAU',axis=1)

df_aggregated.columns = ['_'.join(col).strip() for col in df_aggregated.columns.values]
df_aggregated.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'}, inplace=True)

df_aggregated.shape

/tmp/ipykernel_35/1410330915.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_aggregated = df_aggregated.drop('SK_ID_BUREAU',axis=1)


(305811, 166)

In [70]:
df_aggregated = pd.merge(df_id,df_aggregated,how='left',on='SK_ID_CURR')
df_aggregated

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT_mean,DAYS_CREDIT_count,DAYS_CREDIT_sum,DAYS_CREDIT_max,DAYS_CREDIT_min,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_count,CREDIT_DAY_OVERDUE_sum,...,PERCENTAGE_REIMBURSED_mean,PERCENTAGE_REIMBURSED_count,PERCENTAGE_REIMBURSED_sum,PERCENTAGE_REIMBURSED_max,PERCENTAGE_REIMBURSED_min,NUMBER_LOANS_mean,NUMBER_LOANS_count,NUMBER_LOANS_sum,NUMBER_LOANS_max,NUMBER_LOANS_min
0,100001,5896630,-735.000000,7,-5145,-49,-1572,0.0,7,0,...,0.282518,7,1.977625,0.987405,0.0,7.0,7,49,7,7
1,100002,6158904,-874.000000,8,-6992,-103,-1437,0.0,8,0,...,0.136545,4,0.546180,0.546180,0.0,8.0,8,64,8,8
2,100003,5885877,-1400.750000,4,-5603,-606,-2586,0.0,4,0,...,0.000000,4,0.000000,0.000000,0.0,4.0,4,16,4,4
3,100004,6829133,-867.000000,2,-1734,-408,-1326,0.0,2,0,...,0.000000,2,0.000000,0.000000,0.0,2.0,2,4,2,2
4,100005,6735200,-190.666667,3,-572,-62,-373,0.0,3,0,...,0.601256,3,1.803768,0.954794,0.0,3.0,3,9,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249,5371696,-1667.076923,13,-21672,-483,-2713,0.0,13,0,...,0.090595,10,0.905950,0.905950,0.0,13.0,13,169,13,13
305807,456250,6817236,-862.000000,3,-2586,-760,-1002,0.0,3,0,...,0.575079,3,1.725236,0.870515,0.0,3.0,3,9,3,3
305808,456253,6098498,-867.500000,4,-3470,-713,-919,0.0,4,0,...,0.299308,4,1.197232,0.722132,0.0,4.0,4,16,4,4
305809,456254,6669849,-1104.000000,1,-1104,-1104,-1104,0.0,1,0,...,0.000000,1,0.000000,0.000000,0.0,1.0,1,1,1,1


In [71]:
df_aggregated.replace([np.inf, -np.inf], np.nan, inplace=True)

In [72]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan,strategy = 'constant',fill_value=-1)
df_aggregated = pd.DataFrame(imputer.fit_transform(df_aggregated),columns=df_aggregated.columns)
df_aggregated

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT_mean,DAYS_CREDIT_count,DAYS_CREDIT_sum,DAYS_CREDIT_max,DAYS_CREDIT_min,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_count,CREDIT_DAY_OVERDUE_sum,...,PERCENTAGE_REIMBURSED_mean,PERCENTAGE_REIMBURSED_count,PERCENTAGE_REIMBURSED_sum,PERCENTAGE_REIMBURSED_max,PERCENTAGE_REIMBURSED_min,NUMBER_LOANS_mean,NUMBER_LOANS_count,NUMBER_LOANS_sum,NUMBER_LOANS_max,NUMBER_LOANS_min
0,100001.0,5896630.0,-735.000000,7.0,-5145.0,-49.0,-1572.0,0.0,7.0,0.0,...,0.282518,7.0,1.977625,0.987405,0.0,7.0,7.0,49.0,7.0,7.0
1,100002.0,6158904.0,-874.000000,8.0,-6992.0,-103.0,-1437.0,0.0,8.0,0.0,...,0.136545,4.0,0.546180,0.546180,0.0,8.0,8.0,64.0,8.0,8.0
2,100003.0,5885877.0,-1400.750000,4.0,-5603.0,-606.0,-2586.0,0.0,4.0,0.0,...,0.000000,4.0,0.000000,0.000000,0.0,4.0,4.0,16.0,4.0,4.0
3,100004.0,6829133.0,-867.000000,2.0,-1734.0,-408.0,-1326.0,0.0,2.0,0.0,...,0.000000,2.0,0.000000,0.000000,0.0,2.0,2.0,4.0,2.0,2.0
4,100005.0,6735200.0,-190.666667,3.0,-572.0,-62.0,-373.0,0.0,3.0,0.0,...,0.601256,3.0,1.803768,0.954794,0.0,3.0,3.0,9.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249.0,5371696.0,-1667.076923,13.0,-21672.0,-483.0,-2713.0,0.0,13.0,0.0,...,0.090595,10.0,0.905950,0.905950,0.0,13.0,13.0,169.0,13.0,13.0
305807,456250.0,6817236.0,-862.000000,3.0,-2586.0,-760.0,-1002.0,0.0,3.0,0.0,...,0.575079,3.0,1.725236,0.870515,0.0,3.0,3.0,9.0,3.0,3.0
305808,456253.0,6098498.0,-867.500000,4.0,-3470.0,-713.0,-919.0,0.0,4.0,0.0,...,0.299308,4.0,1.197232,0.722132,0.0,4.0,4.0,16.0,4.0,4.0
305809,456254.0,6669849.0,-1104.000000,1.0,-1104.0,-1104.0,-1104.0,0.0,1.0,0.0,...,0.000000,1.0,0.000000,0.000000,0.0,1.0,1.0,1.0,1.0,1.0


In [73]:
temp = df_balance.copy()

empty_columns=[]

for column in temp.columns:
    if temp[column].isnull().sum()/305811>0.9:
        empty_columns.append(column)

temp= temp.drop(columns=empty_columns,axis=1)

print(empty_columns)
print(len(empty_columns))
temp

[]
0


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [74]:

temp.replace('X',-1,inplace=True)
temp['CLIENT_ON_TIME'] = temp['STATUS'].apply(lambda x: 1 if x=='0' else 0) 
temp['CLIENT_LATE'] = temp['STATUS'].apply(lambda x: 1 if x == '1' else 0)
temp['CLIENT_VERY_LATE'] = temp['STATUS'].apply(lambda x: 1 if (x=='2' or x=='5' or x=='3' or x=='4') else 0)
temp = temp.drop('STATUS',axis=1)


In [75]:
temp.head(50)
temp = temp.groupby(['SK_ID_BUREAU'],as_index=False).agg(['mean','max','min','sum'])


temp.head()

SK_ID_BUREAU MONTHS_BALANCE               CLIENT_ON_TIME              \
                         mean max min   sum           mean max min sum   
0      5001709          -48.0   0 -96 -4656       0.000000   0   0   0   
1      5001710          -41.0   0 -82 -3403       0.060241   1   0   5   
2      5001711           -1.5   0  -3    -6       0.750000   1   0   3   
3      5001712           -9.0   0 -18  -171       0.526316   1   0  10   
4      5001713          -10.5   0 -21  -231       0.000000   0   0   0   

  CLIENT_LATE             CLIENT_VERY_LATE              
         mean max min sum             mean max min sum  
0         0.0   0   0   0              0.0   0   0   0  
1         0.0   0   0   0              0.0   0   0   0  
2         0.0   0   0   0              0.0   0   0   0  
3         0.0   0   0   0              0.0   0   0   0  
4         0.0   0   0   0              0.0   0   0   0

In [76]:
temp.columns = ['_'.join(col).strip() for col in temp.columns.values]
temp

,SK_ID_BUREAU_,MONTHS_BALANCE_mean,MONTHS_BALANCE_max,MONTHS_BALANCE_min,MONTHS_BALANCE_sum,CLIENT_ON_TIME_mean,CLIENT_ON_TIME_max,CLIENT_ON_TIME_min,CLIENT_ON_TIME_sum,CLIENT_LATE_mean,CLIENT_LATE_max,CLIENT_LATE_min,CLIENT_LATE_sum,CLIENT_VERY_LATE_mean,CLIENT_VERY_LATE_max,CLIENT_VERY_LATE_min,CLIENT_VERY_LATE_sum
0,5001709,-48.0,0,-96,-4656,0.000000,0,0,0,0.000000,0,0,0,0.0,0,0,0
1,5001710,-41.0,0,-82,-3403,0.060241,1,0,5,0.000000,0,0,0,0.0,0,0,0
2,5001711,-1.5,0,-3,-6,0.750000,1,0,3,0.000000,0,0,0,0.0,0,0,0
3,5001712,-9.0,0,-18,-171,0.526316,1,0,10,0.000000,0,0,0,0.0,0,0,0
4,5001713,-10.5,0,-21,-231,0.000000,0,0,0,0.000000,0,0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,6842884,-23.5,0,-47,-1128,0.187500,1,0,9,0.000000,0,0,0,0.0,0,0,0
817391,6842885,-11.5,0,-23,-276,0.500000,1,0,12,0.000000,0,0,0,0.5,1,0,12
817392,6842886,-16.0,0,-32,-528,0.242424,1,0,8,0.000000,0,0,0,0.0,0,0,0
817393,6842887,-18.0,0,-36,-666,0.162162,1,0,6,0.000000,0,0,0,0.0,0,0,0


In [81]:
temp.rename = temp.rename(columns={'SK_ID_BUREAU_':'SK_ID_BUREAU'},inplace=True)

In [85]:
df_merged = pd.merge(df_aggregated,temp,on='SK_ID_BUREAU',how='left')

print(df_merged.isnull().sum())

df_merged = pd.DataFrame(imputer.fit_transform(df_merged),columns=df_merged.columns)
df_merged

SK_ID_CURR                    0
SK_ID_BUREAU                  0
DAYS_CREDIT_mean              0
DAYS_CREDIT_count             0
DAYS_CREDIT_sum               0
                          ...  
CLIENT_LATE_sum          171329
CLIENT_VERY_LATE_mean    171329
CLIENT_VERY_LATE_max     171329
CLIENT_VERY_LATE_min     171329
CLIENT_VERY_LATE_sum     171329
Length: 183, dtype: int64


,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT_mean,DAYS_CREDIT_count,DAYS_CREDIT_sum,DAYS_CREDIT_max,DAYS_CREDIT_min,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_count,CREDIT_DAY_OVERDUE_sum,...,CLIENT_ON_TIME_min,CLIENT_ON_TIME_sum,CLIENT_LATE_mean,CLIENT_LATE_max,CLIENT_LATE_min,CLIENT_LATE_sum,CLIENT_VERY_LATE_mean,CLIENT_VERY_LATE_max,CLIENT_VERY_LATE_min,CLIENT_VERY_LATE_sum
0,100001.0,5896630.0,-735.000000,7.0,-5145.0,-49.0,-1572.0,0.0,7.0,0.0,...,0.0,3.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100002.0,6158904.0,-874.000000,8.0,-6992.0,-103.0,-1437.0,0.0,8.0,0.0,...,0.0,18.0,0.045455,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,100003.0,5885877.0,-1400.750000,4.0,-5603.0,-606.0,-2586.0,0.0,4.0,0.0,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,100004.0,6829133.0,-867.000000,2.0,-1734.0,-408.0,-1326.0,0.0,2.0,0.0,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,100005.0,6735200.0,-190.666667,3.0,-572.0,-62.0,-373.0,0.0,3.0,0.0,...,0.0,7.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249.0,5371696.0,-1667.076923,13.0,-21672.0,-483.0,-2713.0,0.0,13.0,0.0,...,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
305807,456250.0,6817236.0,-862.000000,3.0,-2586.0,-760.0,-1002.0,0.0,3.0,0.0,...,0.0,8.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305808,456253.0,6098498.0,-867.500000,4.0,-3470.0,-713.0,-919.0,0.0,4.0,0.0,...,0.0,12.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305809,456254.0,6669849.0,-1104.000000,1.0,-1104.0,-1104.0,-1104.0,0.0,1.0,0.0,...,0.0,8.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
df_merged.isnull().sum()

SK_ID_CURR               0
SK_ID_BUREAU             0
DAYS_CREDIT_mean         0
DAYS_CREDIT_count        0
DAYS_CREDIT_sum          0
                        ..
CLIENT_LATE_sum          0
CLIENT_VERY_LATE_mean    0
CLIENT_VERY_LATE_max     0
CLIENT_VERY_LATE_min     0
CLIENT_VERY_LATE_sum     0
Length: 183, dtype: int64

In [87]:
df_merged.shape

(305811, 183)

In [88]:
from pathlib import Path  
filepath = Path('/kaggle/working/bureau_kaggle.csv')

df_merged.to_csv(filepath, index=False)
